# Model Selection

In the past two sections, we've covered quite a bit of ground in our understanding of Stochastic Block Models. We introduced the Structured Independent Edge Model, a generalization of the Stochastic Block Model that allows us to compare probabilities of various edge clusters in the network. Next, we introduced the community detection problem, and showed how the adjacency spectal embedding coupled with various adjustments of unsupervised clustering techniques allowed us to learn estimated community assignments $\vec z$ fo the nodes of the network. We then learned how we can use these community assignments to produce estimates of the block matrix, $\hat B$, so that we have a useful model of the network.

As we have learned over the course of the book so far, one of our core aims in machine learning is to identify the *simplest* model that maintains a *faithful* representation of the underlying data with which we are presented. While a direct estimate, $\hat B$, of the block matrix might be a faithful representation of the data we are presented with, we might actually be able to simplify $\hat B$ slightly. Let's construct a quick demonstration of what we mean by this. 